In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report

# # Load the Titanic dataset (you can replace this with your dataset)
# data = pd.read_csv("C:/Users/mihir/OneDrive/Documents/Data Sets/Others/Titanic-Dataset.csv")

# # Preprocess the data
# data = data.dropna()  # Remove rows with missing values for simplicity
# X = data[['Pclass', 'Age', 'Fare']]  # Features (you can include more features)
# y = data['Survived']  # Target variable

# # Encode categorical variables (if any)
# # encoder = LabelEncoder()
# # X['sex'] = encoder.fit_transform(X['sex'])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train a Logistic Regression model (you can choose a different model)
# model = LogisticRegression()
# model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = model.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

# print("Accuracy:", accuracy)
# print("Classification Report:\n", report)


In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the Titanic dataset (you can replace this with your dataset)
data = pd.read_csv("C:/Users/mihir/OneDrive/Documents/Data Sets/Others/Titanic-Dataset.csv")

In [ ]:
data.describe()
# titanic_data= data.copy()

In [ ]:
import seaborn as sns
sns.heatmap(data.corr(),cmap='YlGnBu')

In [61]:
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_indices,test_indices in split.split(data,data[["Survived","Pclass","Sex"]]):
    start_train_set= data.loc[train_indices]
    start_test_set= data.loc[test_indices]

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.subplot(1,2,1)
start_train_set["Survived"].hist()
start_train_set['Pclass'].hist()
plt.subplot(1,2,2)
start_test_set["Survived"].hist()
start_test_set['Pclass'].hist()

plt.show()

In [62]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.impute import SimpleImputer

class AgeImputer (BaseEstimator,TransformerMixin):
    def fit(self,x,y=None):
        return self
    def transform(self,x):
        imputer= SimpleImputer(strategy="mean")
        x['Age']=imputer.fit_transform(x[['Age']])
        return x

In [63]:
from sklearn.preprocessing import OneHotEncoder

class FeatureEncoder(BaseEstimator,TransformerMixin):
    def fit(self,x,y=None):
        return self
    def transform(self,x):
        encoder=OneHotEncoder()
        matrix=encoder.fit_transform(x[['Embarked']]).toarray()
        column_names=['C','S','Q','N']

        for i in range(len(matrix.T)):
            x[column_names[i]]=matrix.T[i]
        matrix=encoder.fit_transform(x[['Sex']]).toarray()
        column_names=['Female','Male']

        for i in range(len(matrix.T)):
            x[column_names[i]]=matrix.T[i]
        return x

In [64]:
class FeatureDropper(BaseEstimator,TransformerMixin):
    def fit(self,x,y=None):
        return self
    def transform(self,x):
        return x.drop(['Embarked','Name','Ticket','Cabin','Sex','N'], axis=1,errors="ignore")

In [65]:
from sklearn.pipeline import Pipeline
pipeline= Pipeline([('ageimputer',AgeImputer()),('featureencoder',FeatureEncoder()),('featuredropper',FeatureDropper())])

In [66]:
start_train_set=pipeline.fit_transform(start_train_set)
# start_train_set

In [67]:
start_train_set

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,C,S,Q,Female,Male
646,647,0,3,19.000000,0,0,7.8958,0.0,0.0,1.0,0.0,1.0
42,43,0,3,29.856561,0,0,7.8958,1.0,0.0,0.0,0.0,1.0
77,78,0,3,29.856561,0,0,8.0500,0.0,0.0,1.0,0.0,1.0
656,657,0,3,29.856561,0,0,7.8958,0.0,0.0,1.0,0.0,1.0
279,280,1,3,35.000000,1,1,20.2500,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
574,575,0,3,16.000000,0,0,8.0500,0.0,0.0,1.0,0.0,1.0
34,35,0,1,28.000000,1,0,82.1708,1.0,0.0,0.0,0.0,1.0
837,838,0,3,29.856561,0,0,8.0500,0.0,0.0,1.0,0.0,1.0
639,640,0,3,29.856561,1,0,16.1000,0.0,0.0,1.0,0.0,1.0


In [71]:
from sklearn.preprocessing import StandardScaler

X=start_train_set.drop(['Survived'],axis=1)
y= start_train_set['Survived']
scaler= StandardScaler()
x_data=scaler.fit_transform(X)
y_data= y.to_numpy()

In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf= RandomForestClassifier()
param_grid=[
    {'n_estimators':[10,100,200,500],'max_depth':[None,5,10],'min_samples_split':[2,3,4]}
]

grid_search= GridSearchCV(clf,param_grid,cv=3,scoring='accuracy',return_train_score=True)
grid_search.fit(x_data,y_data)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [None, 5, 10],
                          'min_samples_split': [2, 3, 4],
                          'n_estimators': [10, 100, 200, 500]}],
             return_train_score=True, scoring='accuracy')

In [75]:
final_cf=grid_search.best_estimator_

In [76]:
final_cf

RandomForestClassifier(max_depth=5, min_samples_split=3, n_estimators=500)

In [77]:
start_test_set=pipeline.fit_transform(start_test_set)
start_test_set

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,C,S,Q,Female,Male
540,541,1,1,36.000000,0,2,71.0000,0.0,0.0,1.0,1.0,0.0
493,494,0,1,71.000000,0,0,49.5042,1.0,0.0,0.0,0.0,1.0
797,798,1,3,31.000000,0,0,8.6833,0.0,0.0,1.0,1.0,0.0
863,864,0,3,29.091837,8,2,69.5500,0.0,0.0,1.0,1.0,0.0
90,91,0,3,29.000000,0,0,8.0500,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
234,235,0,2,24.000000,0,0,10.5000,0.0,0.0,1.0,0.0,1.0
138,139,0,3,16.000000,0,0,9.2167,0.0,0.0,1.0,0.0,1.0
676,677,0,3,24.500000,0,0,8.0500,0.0,0.0,1.0,0.0,1.0
102,103,0,1,21.000000,0,1,77.2875,0.0,0.0,1.0,0.0,1.0


In [80]:
X_test=start_test_set.drop(['Survived'],axis=1)
y_test=start_test_set['Survived']
# y_test=

In [81]:
scaler=StandardScaler()
x_data_test= scaler.fit_transform(X_test)
y_data_test=y_test.to_numpy()

In [82]:
final_cf.score(x_data_test,y_data_test)

0.8212290502793296

In [83]:
final_data= pipeline.fit_transform(data)

In [84]:
final_data

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,C,S,Q,Female,Male
0,1,0,3,22.000000,1,0,7.2500,0.0,0.0,1.0,0.0,1.0
1,2,1,1,38.000000,1,0,71.2833,1.0,0.0,0.0,1.0,0.0
2,3,1,3,26.000000,0,0,7.9250,0.0,0.0,1.0,1.0,0.0
3,4,1,1,35.000000,1,0,53.1000,0.0,0.0,1.0,1.0,0.0
4,5,0,3,35.000000,0,0,8.0500,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,13.0000,0.0,0.0,1.0,0.0,1.0
887,888,1,1,19.000000,0,0,30.0000,0.0,0.0,1.0,1.0,0.0
888,889,0,3,29.699118,1,2,23.4500,0.0,0.0,1.0,1.0,0.0
889,890,1,1,26.000000,0,0,30.0000,1.0,0.0,0.0,0.0,1.0


In [85]:
x_final= final_data.drop(['Survived'],axis=1)
y_final=final_data['Survived']
scaler= StandardScaler()
x_data_final=scaler.fit_transform(x_final)
y_data_final= y_final.to_numpy()


In [86]:
prod_clf= RandomForestClassifier()
param_grid=[
    {'n_estimators':[10,100,200,500],'max_depth':[None,5,10],'min_samples_split':[2,3,4]}
]

grid_search= GridSearchCV(prod_clf,param_grid,cv=3,scoring='accuracy',return_train_score=True)
grid_search.fit(x_data_final,y_data_final)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [None, 5, 10],
                          'min_samples_split': [2, 3, 4],
                          'n_estimators': [10, 100, 200, 500]}],
             return_train_score=True, scoring='accuracy')

In [87]:
prod_final_clf= grid_search.best_estimator_
prod_final_clf

RandomForestClassifier(max_depth=5, min_samples_split=3, n_estimators=500)

In [ ]:
# titanic_test_data= pd